# Interactive Phase Diagram

This is an interactive notebook in which the user can click on points in the phase diagram of $E_0$ and $\omega$, and see the corresponding collective coordinate evolution data.

In [ ]:
%matplotlib notebook

import numpy as np
from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec
import sys

sys.path.append('..')
from Utils.Fits import *
from Utils.PhaseDiagramFunctions import *

data_directory = 'CollectiveCoordinateIntegration/CollectiveCoordinateDataSupplied'

R_directory = data_directory + '/R/'
eta_directory = data_directory + '/eta/'
times = np.load(data_directory + '/General/times.npy')

E0_values, omega_values = get_E0_and_omega_arrays(R_directory)

Calculate the phase diagrams. This cell is commented out as the data is already supplied. 

Important: Before this cell is run, you need to have integrated the collective coordinates by running the SageMath script `CCIntegration.sage`.

In [ ]:
# data_directory = 'CollectiveCoordinateIntegration/CollectiveCoordinateData'

# mean_winding_number_array = get_phase_diagram(data_directory, mean_winding_number)
# helicity_criterion_array = get_helicity_criterion(data_directory, mean_winding_number_array)
# energies_during_cycle_array = get_phase_diagram(data_directory, get_mean_energy, total_energy)
# maximum_radius_during_cycle_array = get_phase_diagram(data_directory, max_radius)

# if not os.path.exists(data_directory + '/PhaseDiagramData'):
#     os.mkdir(data_directory + '/PhaseDiagramData')

# np.save(data_directory + '/PhaseDiagramData/MeanWindingNumber', mean_winding_number)
# np.save(data_directory + '/PhaseDiagramData/HelicityCriterion', helicity_criterion_array)
# np.save(data_directory + '/PhaseDiagramData/EnergiesDuringCycle', energies_during_cycle_array)
# np.save(data_directory + '/PhaseDiagramData/MaxR', maximum_radius_during_cycle_array)

# R_directory = data_directory + '/R/'
# eta_directory = data_directory + '/eta/'
# times = np.load(data_directory + '/General/times.npy')

# E0_values, omega_values = get_E0_and_omega_arrays(R_directory)

Load the data used to plot the phase diagram.

In [ ]:
helicity_criterion_array = np.load(data_directory + '/PhaseDiagramData/HelicityCriterion.npy')

Load various parameters obtained during the collective coordinate integration, such as fitting parameters.

In [ ]:
Bz, dw_width, E_exchangeFitParams, E_magnetic_integralFitParams, E_electric_integralFitParams, alpha, F_RexFitParams, Gamma11FitParams, Gamma22FitParams, G12FitParams, Xi1FitParams, Xi2FitParams, Xi3FitParams, Xi4FitParams, optimalSkyrmionRadius = read_parameters(data_directory)

Display the phase diagram.

In [ ]:
fig = plt.figure(constrained_layout=True, figsize=(6, 8))
gs = GridSpec(4, 2, figure=fig)

ax1 = fig.add_subplot(gs[:2, :])
ax2 = fig.add_subplot(gs[2, 0])
ax3 = fig.add_subplot(gs[2, 1])
ax4 = fig.add_subplot(gs[3, 0])
ax5 = fig.add_subplot(gs[3, 1])

im = ax1.imshow(helicity_criterion_array, extent=[omega_values[0], omega_values[-1], E0_values[0], E0_values[-1]], origin='lower', picker=True)

ax1.set_ylabel(r'$E_0$', fontsize=15)
ax1.set_xlabel(r'$\omega$', fontsize=15)

ax1.set_aspect(0.8)

# Placeholder initial point
start_idx = 0
R, eta = get_coordinates(E0_values[start_idx], omega_values[start_idx], data_directory)
Bz_array = np.full_like(times, Bz)
Ez_array = E0_values[start_idx] * np.cos(omega_values[start_idx]*times)

Rplot, = ax2.plot(times, R)
ax2.set_xlabel(r'$t$')
ax2.set_ylabel(r'$R$')

etaplot, = ax3.plot(times, eta)
ax3.set_xlabel(r'$t$')
ax3.set_ylabel(r'$\eta$')

Rcut = R[int(0.8*len(R)):]
etacut = eta[int(0.8*len(eta)):]
timescut = times[int(0.8*len(times)):]
Ez_arraycut = Ez_array[int(0.8*len(eta)):]
Bz_arraycut = Bz_array[int(0.8*len(eta)):]
orbitplot, = ax4.plot(Rcut*np.cos(etacut), Rcut*np.sin(etacut))
ax4.set_xlabel(r'$R\cos\eta$')
ax4.set_ylabel(r'$R\sin\eta$')
ax4.set_aspect(1)

ax4.set_xlim(np.min(Rcut*np.cos(etacut)), np.max(Rcut*np.cos(etacut)))
ax4.set_ylim(np.min(Rcut*np.sin(etacut)), np.max(Rcut*np.sin(etacut)))

energies = total_energy(Rcut, etacut, Bz_arraycut, Ez_arraycut, E_exchangeFitParams, E_magnetic_integralFitParams, E_electric_integralFitParams)
energiesplot, = ax5.plot(timescut, energies)
ax5.set_xlabel(r'$t$')
ax5.set_ylabel(r'$U$')

coordinate_text = ax1.text(-1., 2.2, "E0={:.2f}".format(E0_values[start_idx]) + ',\n' + "omega={:.2f}".format(omega_values[start_idx]))

# Point to mark where on the diagram is being shown
shownpoint, = ax1.plot(omega_values[start_idx], E0_values[start_idx], '.', color='cyan')

def onpick(event):
    
    mouseevent = event.mouseevent
    x_idx = np.argmin(np.abs(omega_values - mouseevent.xdata))
    y_idx = np.argmin(np.abs(E0_values - mouseevent.ydata))
    
    E = E0_values[y_idx]
    omega = omega_values[x_idx]
    R, eta = get_coordinates(E, omega, data_directory)
    Ez_array = E * np.cos(omega*times)
    Rplot.set_data(times, R)
    etaplot.set_data(times, eta)
    ax2.set_ylim(np.min(R), np.max(R))
    ax3.set_ylim(np.min(eta), np.max(eta))
    coordinate_text.set_text('E0={:.2f}'.format(E) + ',\n' + 'omega={:.2f}'.format(omega))
    shownpoint.set_data(omega, E)
    
    Rcut = R[int(0.8*len(R)):]
    etacut = eta[int(0.8*len(eta)):]
    
    orbitplot.set_data(Rcut*np.cos(etacut), Rcut*np.sin(etacut))
    ax4.set_xlim(np.min(Rcut*np.cos(etacut)), np.max(Rcut*np.cos(etacut)))
    ax4.set_ylim(np.min(Rcut*np.sin(etacut)), np.max(Rcut*np.sin(etacut)))
    ax4.set_aspect(1)
    
    energies = total_energy(Rcut, etacut, Bz_arraycut, Ez_arraycut, E_exchangeFitParams, E_magnetic_integralFitParams, E_electric_integralFitParams)
    energiesplot.set_data(timescut, energies)
    ax5.set_ylim(np.min(energies), np.max(energies))
    
    Rcut = R[len(R)//2:]
    etacut = eta[len(eta)//2:]
    
fig.canvas.mpl_connect('pick_event', onpick)